# PaddlePaddle BYOS

## Pre-requisites

This notebook shows how to use the SageMaker Python SDK to run your code in a local container before deploying to SageMaker's managed training or hosting environments.  This can speed up iterative testing and debugging while using the same familiar Python SDK interface.  Just change your estimator's `train_instance_type` to `local` (or `local_gpu` if you're using an ml.p2 or ml.p3 notebook instance).

In order to use this feature you'll need to install docker-compose (and nvidia-docker if training with a GPU).

**Note, you can only run a single local notebook at one time.**

In [1]:
# !/bin/bash ./utils/setup.sh

## Overview

The **SageMaker Python SDK** helps you deploy your models for training and hosting in optimized, productions ready containers in SageMaker. The SageMaker Python SDK is easy to use, modular, extensible and compatible with TensorFlow, MXNet, PyTorch and Chainer. This tutorial focuses on how to create a convolutional neural network model to train the [Cifar10 dataset](https://www.cs.toronto.edu/~kriz/cifar.html) using **PyTorch in local mode**.

### Set up the environment

This notebook was created and tested on a single ml.p2.xlarge notebook instance.

Let's start by specifying:

- The S3 bucket and prefix that you want to use for training and model data. This should be within the same region as the Notebook Instance, training, and hosting.
- The IAM role arn used to give training and hosting access to your data. See the documentation for how to create these. Note, if more than one role is required for notebook instances, training, and/or hosting, please replace the sagemaker.get_execution_role() with appropriate full IAM role arn string(s).

In [2]:
import os
import sagemaker

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix = 'sagemaker/DEMO-PaddleNLP'

role = sagemaker.get_execution_role()

In [3]:
# import subprocess

# instance_type = 'local'

# if subprocess.call('nvidia-smi') == 0:
#     ## Set type to GPU if one is present
#     instance_type = 'local_gpu'
    
# print("Instance type = " + instance_type)

# Prepare data

In [4]:
with open('../train/all.txt') as f:
    data = f.readlines()

split = 0.7
split_index = int(len(data) * split)
training = data[:split_index]
testing = data[split_index:]

# open file in write mode
with open(r'../train/train.txt', 'w') as fp:
    for item in training:
        # write each item on a new line
        fp.write("%s\n" % item)
    print('Done')

with open(r'../train/dev.txt', 'w') as fp:
    for item in testing:
        # write each item on a new line
        fp.write(item)
    print('Done')

Done
Done


### Upload the data
We use the ```sagemaker.Session.upload_data``` function to upload our datasets to an S3 location. The return value inputs identifies the location -- we will use this later when we start the training job.

In [6]:
data_location = sagemaker.Session().upload_data(path = "../train", key_prefix=prefix)
# base_dir = 'file:///home/ec2-user/SageMaker/paddlenlp_sagemaker/data/'
# inputs = {'training': base_dir}
# print(inputs)

In [7]:
data_location

's3://sagemaker-us-east-1-726335585155/sagemaker/DEMO-PaddleNLP'

## Script Functions

SageMaker invokes the main function defined within your training script for training. When deploying your trained model to an endpoint, the model_fn() is called to determine how to load your trained model. The model_fn() along with a few other functions list below are called to enable predictions on SageMaker.

### [Predicting Functions](https://github.com/aws/sagemaker-pytorch-containers/blob/master/src/sagemaker_pytorch_container/serving.py)
* model_fn(model_dir) - loads your model.
* input_fn(serialized_input_data, content_type) - deserializes predictions to predict_fn.
* output_fn(prediction_output, accept) - serializes predictions from predict_fn.
* predict_fn(input_data, model) - calls a model on data deserialized in input_fn.

The model_fn() is the only function that doesn't have a default implementation and is required by the user for using PyTorch on SageMaker. 

## Create a training job using the sagemaker.PyTorch estimator

The `PyTorch` class allows us to run our training function on SageMaker. We need to configure it with our training script, an IAM role, the number of training instances, and the training instance type. For local training with GPU, we could set this to "local_gpu".  In this case, `instance_type` was set above based on your whether you're running a GPU instance.

After we've constructed our `PyTorch` object, we fit it using the data we uploaded to S3. Even though we're in local mode, using S3 as our data source makes sense because it maintains consistency with how SageMaker's distributed, managed training ingests data.


## SageMaker Training using GPU instance

In [8]:
inputs = {'training': data_location}

print(inputs)

{'training': 's3://sagemaker-us-east-1-726335585155/sagemaker/DEMO-PaddleNLP'}


In [9]:
#upload uie-base-en pretrain

uie_en_model_s3 = sagemaker.Session().upload_data(path = "../uie-base-en/taskflow/information_extraction/uie-base-en", key_prefix="model_uie_base_en")

In [10]:
from sagemaker.pytorch import PyTorch

hyperparameters = {'train_path': '/opt/ml/input/data/training/train.txt', 
                   'dev_path': '/opt/ml/input/data/training/dev.txt', 
                   'save_dir': '/opt/ml/model', 
                   'learning_rate': 1e-5, 
                   'batch_size': 16, 
                   'max_seq_len':512, 
                   'num_epochs': 1, 
                   'model': 'uie-base',
                   'seed': 1000,
                   'logging_steps': 10,
                   'valid_steps': 100,
                   'device': 'gpu'}

instance_type = 'ml.p3.2xlarge'  # 'ml.p3.2xlarge' or 'ml.p3.8xlarge' or ...

#git_config = {'repo': 'https://github.com/whn09/paddlenlp_sagemaker.git', 'branch': 'main'}

estimator = PyTorch(entry_point='finetune.py',
                    source_dir='./',
                           # git_config=git_config,
                    role=role,
                    hyperparameters=hyperparameters,
                    framework_version='1.9.1',
                    py_version='py38',
                    script_mode=True,
                    instance_count=1,  # 1 or 2 or ...
                    instance_type=instance_type,
                    # Parameters required to enable checkpointing
                    checkpoint_s3_uri=uie_en_model_s3, #使用你自己用来保存/加载模型的s3桶地址, 注意桶需要在us-east-1
                    checkpoint_local_path="/opt/ml/checkpoints")

estimator.fit(inputs)

2022-09-09 03:45:25 Starting - Starting the training job...ProfilerReport-1662695125: InProgress
...
2022-09-09 03:46:09 Starting - Preparing the instances for training.........
2022-09-09 03:47:50 Downloading - Downloading input data...
2022-09-09 03:48:10 Training - Downloading the training image..........................bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2022-09-09 03:52:35,846 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2022-09-09 03:52:35,883 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2022-09-09 03:52:35,891 sagemaker_pytorch_container.training INFO     Invoking user training script.
2022-09-09 03:52:36,461 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
/opt/conda/bin/python -m pip install -r requirements.txt
Looking in indexes: https://opentuna.cn/pypi/web/simple/

2022-09-09 0

UnexpectedStatusException: Error for Training job pytorch-training-2022-09-09-03-45-25-415: Failed. Reason: AlgorithmError: ExecuteUserScriptError:
ExitCode 1
ErrorMessage "TypeError: string indices must be integers
"
Command "/opt/conda/bin/python finetune.py --batch_size 16 --dev_path /opt/ml/input/data/training/dev.txt --device gpu --learning_rate 1e-05 --logging_steps 10 --max_seq_len 512 --model uie-base --num_epochs 1 --save_dir /opt/ml/model --seed 1000 --train_path /opt/ml/input/data/training/train.txt --valid_steps 100", exit code: 1

In [ ]:
training_job_name = estimator.latest_training_job.name
# training_job_name = 'xxx'
print(training_job_name)

# Deploy the trained model to prepare for predictions

The deploy() method creates an endpoint (in this case locally) which serves prediction requests in real-time.

In [ ]:
!rm -rf model.tar.gz
!rm -rf model_*
!rm -rf inference.*
!aws s3 cp s3://$bucket/$training_job_name/output/model.tar.gz .
!tar -xvf model.tar.gz

In [ ]:
!cp inference.* model/
!cp model/code/requirements_gpu.txt model/code/requirements.txt
!cd model && tar -czvf ../model-inference-gpu.tar.gz *

!aws s3 cp model-inference-gpu.tar.gz s3://$bucket/$training_job_name/output/model-inference-gpu.tar.gz

In [ ]:
# instance_type = 'local'
# instance_type = 'ml.m5.xlarge'
instance_type = 'ml.g4dn.xlarge'

# predictor = estimator.deploy(initial_instance_count=1, instance_type=instance_type)

from sagemaker.pytorch.model import PyTorchModel

pytorch_model = PyTorchModel(model_data='s3://{}/{}/output/model-inference-gpu.tar.gz'.format(bucket, training_job_name), role=role,
                             entry_point='infer_gpu.py', framework_version='1.9.0', py_version='py38', model_server_workers=4)  # TODO [For GPU], model_server_workers=6

predictor = pytorch_model.deploy(instance_type=instance_type, initial_instance_count=1)

In [ ]:
# # endpoint_name = 'pytorch-inference-2022-07-05-07-28-16-183'  # m5.2xlarge
# # endpoint_name = 'pytorch-inference-2022-07-06-04-02-11-091'  # g4dn.xlarge, 6 threads
# endpoint_name = 'pytorch-inference-2022-07-06-06-19-21-855'  # g4dn.xlarge, 4 threads
# predictor = sagemaker.predictor.Predictor(endpoint_name=endpoint_name)

# Invoking the endpoint

In [ ]:
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer

predictor.serializer = JSONSerializer()
predictor.deserializer = JSONDeserializer()

In [ ]:
texts = ['"北京市海淀区人民法院\n民事判决书\n(199x)建初字第xxx号\n原告：张三。\n委托代理人李四，北京市 A律师事务所律师。\n被告：B公司，法定代表人王五，开发公司总经理。\n委托代理人赵六，北京市 C律师事务所律师。"', 
         '原告赵六，2022年5月29日生\n委托代理人孙七，深圳市C律师事务所律师。\n被告周八，1990年7月28日出生\n委托代理人吴九，山东D律师事务所律师']

import time
start = time.time()
outputs = predictor.predict(texts)
end = time.time()
print('outputs: ', outputs)
print('time:', end-start)

# for i in range(1000):
#     start = time.time()
#     outputs = predictor.predict(texts)
#     end = time.time()
#     print('time:', end-start)

# Clean-up

Deleting the local endpoint when you're finished is important since you can only run one local endpoint at a time.

In [ ]:
# estimator.delete_endpoint()
predictor.delete_endpoint()